In [ ]:
# Import from local package
import sys
sys.path.append('../model')

from tqdm import tqdm_notebook as progress_bar

import torch

In [ ]:
from models import SimpleCNN3Layer as Model
from collectdata import DataCollector
from plots import plot_truth_vs_predict
from efficiency import efficiency, ValueSet
from training import select_gpu

In [ ]:
# Device configuration
device = select_gpu(0)
#device = torch.device("cpu")

In [ ]:
collector = DataCollector(
    "/data/schreihf/PvFinder/Aug_15_140000.npz",
    120_000, 10_000)

In [ ]:
validation = collector.get_validation(1, device=device)

In [ ]:
model = Model().to(device)

In [ ]:
model.load_state_dict(torch.load('../model/output3/20180815_120000_3layer_30.pyt'))
model.eval()

In [ ]:
%%time
with torch.no_grad():
    outputs = model(validation.dataset.tensors[0]).cpu().numpy()
    labels = validation.dataset.tensors[1].cpu().numpy()

In [ ]:
%%time
total = ValueSet(0,0,0,0)

for label, output in zip(labels, outputs):
    total += efficiency(label, output, 5., 1e-2, .2, 3)

In [ ]:
print(total)
print()
print(total.pretty())

# Some examples of the counting algorithm:

In [ ]:
for i in range(20):
    result = efficiency(labels[i], outputs[i], 5., 1e-2, .2, 3)
    print(result)

In [ ]:
for i in range(20):
    result = efficiency(labels[i], outputs[i], 5., 1e-2,.2, 3)
    ax = plot_truth_vs_predict(labels[i], outputs[i])
    ax.set_title(str(result))